In [ ]:
import numpy as np
from scipy.stats import multivariate_normal
X = np.loadtxt("seeds_dataset.txt")
X = np.delete(X,-1,axis=1)
N = X.shape[0]
# K-Means
def K_Means(X, m):
    C = X[m] # randomize initial clusters
    R = np.zeros((N,3)) # assignment matrix
    while True:
        R_prev = np.copy(R)
        for i in range(N): # assignment
            R[i] = np.array([0,0,0])
            R[i][np.argmin(np.sum((C-X[i])**2, axis=1))] = 1
        if np.array_equal(R,R_prev): # if assignment doesn't update, terminate
            break
        for i in range(3): # refitting
            if np.sum(R[:,i]) != 0:
                C[i,:] = np.mean(X[np.where(R[:,i]==1)],axis=0)
    return R, C

Gaussian Mixture Model

In [ ]:
def GMM(X, n):
    Mu = X[n] # randomized initial cluster mean
    Sigma = np.array([np.identity(7)]*3) # initial convariance matrices
    W = np.random.rand(3,)
    Pi = W/np.sum(W) # randomized initial weight
    Gamma = np.zeros((N,3)) # posterior prob matrix
    K = 0
    while K < 15:
        for i in range(N): # E-step
            sum = 0
            for m in range(3):
                sum += Pi[m]*multivariate_normal.pdf(x = X[i],mean = Mu[m],cov = Sigma[m])
            for k in range(3): # update posterior prob matrix
                Gamma[i][k] = Pi[k]*multivariate_normal.pdf(x = X[i],mean = Mu[k],cov = Sigma[k])/sum
        for k in range(3): # M-step, update parameters
            N_k = np.sum(Gamma[:,k])
            Mu[k] = (X.T @ Gamma[:,k])/N_k
            outer = np.zeros((X.shape[1],X.shape[1]))
            for n in range(N):
                outer = outer + Gamma[n][k] * np.outer(X[n]-Mu[k],X[n]-Mu[k])
            Sigma[k] = outer/N_k
            Pi[k] = N_k/N
        K += 1
    L = np.argmax(Gamma, axis=1) # assignment array
    R = np.zeros((N,3)) # assignment matrix
    R[np.where(L==0),0] = 1
    R[np.where(L==1),1] = 1
    R[np.where(L==2),2] = 1
    return R, Mu, Sigma, Pi

Silhouette coefficient (short for SC)

In [ ]:
def Silhouette_coeff(X, R): # R: assignment matrix
    S = np.zeros((N,)) # SC matrix
    for i in range(N):
        k = np.argmax(R[i]) # cluster index for this sample
        # a: mean distance between this sample and all other samples in the same cluster
        a = sum((np.sum((X[np.where(R[:,k]==1)]-X[i,:])**2,axis=1))**(1/2))/(np.sum(R[:,k])-1)
        # b: mean distance between this sample and all other points in the next nearest cluster
        if k == 0:
            b = min(np.mean(np.sum((X[np.where(R[:,1]==1)] - X[i,:])**2,axis=1)**(1/2)),np.mean(np.sum((X[np.where(R[:,2]==1)] - X[i,:])**2,axis=1)**(1/2)))
        if k == 1:
            b = min(np.mean(np.sum((X[np.where(R[:,0]==1)] - X[i,:])**2,axis=1)**(1/2)),np.mean(np.sum((X[np.where(R[:,2]==1)] - X[i,:])**2,axis=1)**(1/2)))
        if k == 2:
            b = min(np.mean(np.sum((X[np.where(R[:,0]==1)] - X[i,:])**2,axis=1)**(1/2)),np.mean(np.sum((X[np.where(R[:,1]==1)] - X[i,:])**2,axis=1)**(1/2)))
        S[i] = (b-a)/max(a,b) # SC for this sample
    return np.mean(S)

Rand index

In [ ]:
def Rand_index(X, R1, R2): # R1, R2: assignment matrix
    a = 0
    b = 0
    for i in range(N):
        for j in range(i+1,N):
            # if the pair is in both the same cluster in R1 and R2
            if np.array_equal(R1[i],R1[j]) and np.array_equal(R2[i],R2[j]):
                a += 1
            # if the pair is both not in the same cluster in R1 and R2
            if (not np.array_equal(R1[i],R1[j])) and (not np.array_equal(R2[i],R2[j])):
                b += 1
    return 2*(a+b)/(N*(N-1))

In [ ]:
def main():
    K = 5
    KM_SC = []
    GMM_SC = []    ## record the evaluation scores
    RandInd = []
    X_Ind = np.array(range(N))
    for i in range(K):
        m = np.random.choice(X_Ind,size = 3,replace = False)
        R1, C = K_Means(X, m)
        R2, Mu, Sigma, Pi = GMM(X, m)
        KM_SC.append(Silhouette_coeff(X, R1))
        GMM_SC.append(Silhouette_coeff(X, R2))
        RandInd.append(Rand_index(X, R1, R2))
    print("The mean score of Silhouette coefficients for K-Means is ",np.mean(KM_SC)," and standard deviation is ",np.std(KM_SC),";\n",\
    "The mean score of Silhouette coefficients for GMM is ",np.mean(GMM_SC)," and standard deviation is ",np.std(GMM_SC),";\n",\
    "The mean of Rand Indices for (K-Means,GMM) is ",np.mean(RandInd)," and standard deviation is ",np.std(RandInd),".",sep='')

In [ ]:
main()